In [62]:
import getpass
import os

if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your token: ")
if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")


In [91]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain.chat_models import init_chat_model
chat_model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")


In [92]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_prompt = SystemMessagePromptTemplate.from_template(
    "you are a scientific writer"
)

user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a simplified version for high school students from this paper 
    ----
    {paper}
    ----
    then, Generate a creative title for the simplified paper. Also extract related terms and research areas
    
    """,
    input_variables=['paper']
)


In [93]:
user_prompt.format(paper="TEST STRING").pretty_print()

================================ Human Message =================================

You are tasked with creating a simplified version for high school students from this paper 
    ----
    TEST STRING
    ----
    then, Generate a creative title for the simplified paper. Also extract related terms and research areas

    


In [94]:
paper = """ 
Title: The Role of Gut Microbiota in Human Health and Disease

Abstract:
The human gastrointestinal tract harbors a complex and dynamic population of microorganisms known as the gut microbiota. 
These microbes perform essential functions including aiding digestion, regulating immune responses, and synthesizing vitamins. 
Disruptions to this microbial community, termed dysbiosis, have been linked to a range of diseases such as obesity, type 2 diabetes, inflammatory bowel disease, and even neurological disorders. 
Recent advances in sequencing technologies have enabled researchers to better understand the diversity and functional capacity of the gut microbiome, opening avenues for novel therapeutic strategies such as probiotics, prebiotics, and fecal microbiota transplantation.
"""

In [95]:
from langchain.prompts import ChatPromptTemplate
first_prompt = ChatPromptTemplate.from_messages([system_prompt,user_prompt])

In [96]:
chain_one = (
    {"paper" : lambda x: x["paper"]}
    | first_prompt
    | chat_model
    |{"simplified_paper": lambda x: x.content}
)


In [ ]:
simplified_paper = chain_one.invoke({"paper": paper})

In [ ]:
print(simplified_paper)

{'simplified_paper': 'Okay, here\'s a simplified version of that abstract, geared towards high school students:\n\n**Title: Your Gut Buddies: How Bacteria in Your Stomach Affect Your Health**\n\n**Simplified Explanation:**\n\nImagine your stomach and intestines as a bustling city filled with trillions of tiny residents – bacteria, viruses, and fungi.  This community is called the **gut microbiota** (or gut microbiome). These aren\'t just freeloaders; they\'re helpful workers!\n\n*   **They help you digest food:** Think of them as breaking down complex foods your body can\'t handle on its own, like certain types of fiber.\n*   **They train your immune system:** They help your body learn to tell the difference between friend (good bacteria) and foe (harmful invaders), keeping you healthy.\n*   **They make vitamins:** Some bacteria actually produce vitamins like Vitamin K and some B vitamins, which your body needs.\n\nWhen this community of microbes is balanced and happy, everything runs 

In [97]:
from pydantic import BaseModel, Field

class Paper(BaseModel):
    original_paper: str = Field(description="The original scientific paper")
    simplified_paper: str = Field(description="The simplified scientific paper")
    simplified_paper_title: str = Field(description=(
        "The title for the simplified paper"
    ))

In [98]:
structured_llm = chat_model.with_structured_output(Paper)

In [99]:
chain_three = (
    {"paper": lambda x: x["paper"]}
    | first_prompt
    | structured_llm
    | {
        "original_paper": lambda x: x.original_paper,
        "simplified_paper": lambda x: x.simplified_paper,
        "simplified_paper_title": lambda x: x.simplified_paper_title
    }
)

In [100]:
out = chain_three.invoke({"paper": paper})


In [104]:
import textwrap

WRAP_WIDTH = 150  # Adjust based on your terminal size

def wrapped_print(title, content):
    print(f"{title}\n")
    print(textwrap.fill(content, width=WRAP_WIDTH))
    print("\n" + "-" * WRAP_WIDTH)

print("\n" + "=" * WRAP_WIDTH)
print("🔍 Output from chain_three:")
print("=" * WRAP_WIDTH + "\n")

wrapped_print("📄 Original Paper:", out["original_paper"])
wrapped_print("📝 Simplified Paper Title:", out["simplified_paper_title"])
wrapped_print("🔬 Simplified Paper:", out["simplified_paper"])

print("=" * WRAP_WIDTH)



🔍 Output from chain_three:

📄 Original Paper:

The human gastrointestinal tract harbors a complex and dynamic population of microorganisms known as the gut microbiota.  These microbes perform
essential functions including aiding digestion, regulating immune responses, and synthesizing vitamins.  Disruptions to this microbial community,
termed dysbiosis, have been linked to a range of diseases such as obesity, type 2 diabetes, inflammatory bowel disease, and even neurological
disorders.  Recent advances in sequencing technologies have enabled researchers to better understand the diversity and functional capacity of the gut
microbiome, opening avenues for novel therapeutic strategies such as probiotics, prebiotics, and fecal microbiota transplantation.

------------------------------------------------------------------------------------------------------------------------------------------------------
📝 Simplified Paper Title:

Your Gut Buddies: How Bacteria Affect Your Health

--------